In [1]:
from src.daily_notes_processor import DailyNotesProcessor
from src.config import Config
from src.audio_processor import AudioProcessor
from src.note_generator import NoteGenerator
from src.timeline_generator import TimelineGenerator
from faster_whisper import WhisperModel, BatchedInferencePipeline

c:\Users\nicco\OneDrive\Documenti\Obsidian\.venv\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\nicco\OneDrive\Documenti\Obsidian\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step by Step

In [2]:
config = Config('config.yaml')

note_generator = NoteGenerator(
            config.openai_api_key,
            config.config_data['processing']['gpt_model'],
            config.config_data['processing']['temperature']
        )

timeline_generator = TimelineGenerator(
            config,
            config.openai_api_key,
            config.config_data['processing']['gpt_model'],
            config.config_data['processing']['temperature']
        )

audio_processor = AudioProcessor(config)

Loading Fast-Whisper model 'turbo'...
✓ Whisper model loaded successfully


In [3]:
supported_formats = config.config_data['audio']['supported_formats']
audio_files = []

for format_ext in supported_formats:
    audio_files.extend(config.audio_input_path.glob(f"*{format_ext}"))
    
sorted(audio_files)
audio_files

[WindowsPath('c:/Users/nicco/OneDrive/Documenti/Obsidian/daily_notes/AudioInbox/test_record.m4a')]

In [4]:
audio_path = audio_files[0]

# 1. Transcribe audio
transcript_data = audio_processor.transcribe(audio_path)
print(f"Transcription completed ({len(transcript_data['text'])} chars)")


Transcribing: test_record.m4a
✓ Audio validation passed: Valid audio file: 155.0s
Normalizing audio for optimal transcription...
✓ Audio normalized
Starting transcription...


100%|██████████| 6/6 [00:58<00:00,  9.83s/it]

✓ Transcription completed
Transcription completed (1241 chars)


In [8]:
# 2. Get available projects
available_projects = config.get_available_projects()
print(f"Available projects: {', '.join(available_projects)}")

# 3. Generate daily note with project detection
note_path = note_generator.create_daily_note(
    transcript_data=transcript_data,
    available_projects=available_projects,
    audio_filename=audio_path.name,
    output_path=config.daily_notes_path
)

# 4. Delete audio file if configured
if config.config_data['audio']['delete_after_processing']:
    success = audio_processor.delete_audio_file(audio_path)
    if not success:
        print(f"⚠ Warning: Could not delete {audio_path.name}")

print(f"✅ Success! Note: {note_path.name}")

Available projects: ENPICOM, Saliency
ChatCompletion(id='chatcmpl-BeFtknkoH2DEpzvmZUkzjzkaKR9Jl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "project": "Saliency",\n  "summary": "- Today I worked on my main project, Palency, which I believe is a reference to Saliency.\\n- I continued conducting experiments to test the gradient flow on the prediction model over the input embeddings.\\n- I focused on debugging an issue from Friday that was causing zero gradient in certain regions of the antibody sequences.\\n- The problem was related to how sequences were fragmented by the anarchy sequence segment function.\\n- I discovered that some sequences weren\'t properly segmented, particularly in the framework region 4.\\n- This segmentation issue was affecting the indexing and lists, leading to inaccurate results.\\n- I made adjustments to make the process resistant to incorrectly fragmented sequences.\\n- Despite the fixes,

## Interactvie

In [2]:
processor = DailyNotesProcessor('config.yaml')

Loading Whisper model 'medium'...
✓ Whisper model loaded successfully
Setup complete. Drop audio files in: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox


In [3]:
processor.run_interactive() 

🎯 Daily Notes Processor - Interactive Mode

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

🎤 Recording Voice Note
------------------------------

🎙️  Ready to record to: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
🔴 Recording started using: System Default (Microphone (2- EDIFIER W830NB))
Press SPACE to stop or Ctrl+C to cancel...

⏹️  Stopping recording...
✅ Recording stopped! Duration: 245.18 seconds
✅ Audio saved: voice_note_20250603_172430.wav (21118.0 KB)

✅ Recording saved: voice_note_20250603_172430.wav

🔄 Processing voice_note_20250603_172430.wav...

Processing: voice_note_20250603_172430.wav
Transcribing: voice_note_20250603_172430.wav
✓ Audio validation passed: Valid audio file: 245.2s
Normalizing audio for optimal transcription...
✓ Audio normalized
Starting transcription...


c:\Users\nicco\OneDrive\Documenti\Obsidian\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


100%|██████████| 24517/24517 [02:40<00:00, 152.40frames/s]


✓ Transcription completed
Transcription completed (2239 chars)
Available projects: ENPICOM, Saliency
Daily note exists, creating: 2025-06-03_Saliency_173143.md
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-03_Saliency_173143.md
✅ Success! Note: 2025-06-03_Saliency_173143.md
✅ Voice note processed successfully!

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

🎤 Recording Voice Note
------------------------------

🎙️  Ready to record to: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
🔴 Recording started using: System Default (Microphone (2- EDIFIER W830NB))
Press SPACE to stop or Ctrl+C to cancel...

⏹️  Stopping recording...
✅ Recording stopped! Duration: 203.06 seconds
✅ Audio saved: voice_note_20250603_173505.wav (17490.0 KB)

✅ Recording saved: voice_note_20250603_173505.w

100%|██████████| 20305/20305 [01:27<00:00, 232.37frames/s]


✓ Transcription completed
Transcription completed (1519 chars)
Available projects: ENPICOM, Saliency
Created daily note for project 'Saliency': c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes\2025-06-03_Saliency.md
✅ Success! Note: 2025-06-03_Saliency.md
✅ Voice note processed successfully!

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure audio device
4. 📋 Show current settings
5. 📅 Generate timeline
6. 🚪 Exit

📋 Current Settings
------------------------------
Projects Path: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects
Audio Inbox: c:\Users\nicco\OneDrive\Documenti\Obsidian\daily_notes\AudioInbox
Daily Notes: c:\Users\nicco\OneDrive\Documenti\Obsidian\Vault\1. Projects\Daily Notes
Delete after processing: False
Audio Device: System Default (Microphone (2- EDIFIER W830NB))
Available Projects: ENPICOM, Saliency

Choose an option:
1. 📁 Scan for new audio files
2. 🎤 Record new voice note
3. ⚙️  Configure a

In [ ]:
print(f"\n🎤 Recording Voice Note") 
print("-" * 30)

# Test default device first
if not processor.audio_recorder.test_default_device():
    print("⚠️  Default audio device not available.")
    device_id = processor.audio_recorder.select_device()
    processor.audio_recorder.selected_device_id = device_id


🎤 Recording Voice Note
------------------------------
